In [83]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import torch.optim as optim
import math


In [35]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    print('GPU avaialble')
else:
    print('no')

GPU avaialble


In [33]:
data = pd.read_csv('/Users/prathikkundaragi/Downloads/NIFTY 50_Historical_PR_01011990to11102024.csv',index_col='Date')[['Close']]
data.index = pd.to_datetime(data.index)

data

,Close
Date,
2024-10-11,24964.25
2024-10-10,24998.45
2024-10-09,24981.95
2024-10-08,25013.15
2024-10-07,24795.75
...,...
1990-07-10,288.69
1990-07-09,289.69
1990-07-06,289.04


In [38]:
data=data[::-1]
data_float =data['Close'].values.astype(float)
data_float

array([  279.02,   284.04,   289.04, ..., 24981.95, 24998.45, 24964.25])

In [39]:
test_data_size= 30
data_test = data_float[-test_data_size:]
data_train = data_float[:-test_data_size]

print(data_test)

print(data_train)

[25235.9  25278.7  25279.85 25198.7  25145.1  24852.15 24936.4  25041.1
 24918.45 25388.9  25356.5  25383.75 25418.55 25377.55 25415.8  25790.95
 25939.05 25940.4  26004.15 26216.05 26178.95 25810.85 25796.9  25250.1
 25014.6  24795.75 25013.15 24981.95 24998.45 24964.25]
[  279.02   284.04   289.04 ... 25017.75 25052.35 25151.95]


In [ ]:
print(len(data_test))

print(len(data_train))

30
8295


In [42]:
# normalising the data 
scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(data_train.reshape(-1, 1))
train_data_normalized

array([[-1.        ],
       [-0.99959635],
       [-0.9991943 ],
       ...,
       [ 0.98920915],
       [ 0.99199129],
       [ 1.        ]])

In [55]:
#numpy to tensor
train_data_tensor= torch.Tensor(train_data_normalized).view(-1)

In [56]:
#creating sequential data for LSTM 
def create_inout_sequences(data, window=30):
    input_seq = []
    L = len(data)
    for i in range(L-window):
        train_seq = data[i:i+window]
        train_label = data[i+window:i+window+1]
        input_seq.append((train_seq ,train_label))
    return input_seq

In [57]:
#traingn sequence input

train_sequence_input = create_inout_sequences(train_data_tensor)
train_sequence_input

[(tensor([-1.0000, -0.9996, -0.9992, -0.9991, -0.9992, -0.9989, -0.9986, -0.9986,
          -0.9993, -0.9987, -0.9985, -0.9982, -0.9981, -0.9966, -0.9964, -0.9959,
          -0.9945, -0.9953, -0.9958, -0.9965, -0.9962, -0.9956, -0.9944, -0.9943,
          -0.9945, -0.9940, -0.9932, -0.9927, -0.9928, -0.9915]),
  tensor([-0.9915])),
 (tensor([-0.9996, -0.9992, -0.9991, -0.9992, -0.9989, -0.9986, -0.9986, -0.9993,
          -0.9987, -0.9985, -0.9982, -0.9981, -0.9966, -0.9964, -0.9959, -0.9945,
          -0.9953, -0.9958, -0.9965, -0.9962, -0.9956, -0.9944, -0.9943, -0.9945,
          -0.9940, -0.9932, -0.9927, -0.9928, -0.9915, -0.9915]),
  tensor([-0.9915])),
 (tensor([-0.9992, -0.9991, -0.9992, -0.9989, -0.9986, -0.9986, -0.9993, -0.9987,
          -0.9985, -0.9982, -0.9981, -0.9966, -0.9964, -0.9959, -0.9945, -0.9953,
          -0.9958, -0.9965, -0.9962, -0.9956, -0.9944, -0.9943, -0.9945, -0.9940,
          -0.9932, -0.9927, -0.9928, -0.9915, -0.9915, -0.9915]),
  tensor([-0.9903]))

In [91]:
#building LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size= 1, hidden_layer_size= 32,output_size=1,dropout=0.5):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.input_size =input_size
        self.out_size = output_size
        self.num_layers = 2
            
        self.lstm_model= nn.LSTM(hidden_size=self.hidden_layer_size, 
                                     input_size=self.input_size,
                                     num_layers=self.num_layers,                    
                                     dropout=dropout,                              
                                     batch_first=False)
            
        # Final linear layer mapping last hidden vector to desired output size
        self.linear = nn.Linear(self.hidden_layer_size, output_size)
        
    def forward(self, input_seq):
        x= input_seq.view(len(input_seq),1,1)
        seq_len = x.size(0)                             
        batch_size = x.size(1) 
        
        
        #initialisation for hidden  and candidate memory
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_layer_size)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_layer_size)
        
        # Forward pass through stacked LSTM
        lstm_out, _ = self.lstm(x, (h0, c0)) 
        
        # Take last time step’s output (top layer)
        last_hidden = lstm_out[-1]              # (batch, hidden_size)

        # Map to prediction
        out = self.linear(last_hidden)          # (batch, output_size)
        return out.squeeze()      
        

In [93]:
model = LSTM(input_size=1, hidden_layer_size=64, output_size=1, dropout=0.3)
loss_fn = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=1e-3)

In [102]:
for ep in range(60):
    model.train()
    opt.zero_grad()
    yhat = model(train_sequence_input)                
    loss = loss_fn(yhat, train_data_normalized)
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    opt.step()

    model.eval()
    with torch.no_grad():
        val_loss = loss_fn(model(X_val), y_val).item()
    print(f"epoch {ep:03d} | train {loss.item():.6f} | val {val_loss:.6f}")

AttributeError: 'list' object has no attribute 'view'

In [ ]:
@torch.no_grad()
def forecast_rolling(model, last_window_scaled, steps=10):
    model.eval()
    w = last_window_scaled.view(-1).clone()     # (window,)
    preds = []
    for _ in range(steps):
        x = w.view(-1,1,1)                      # (T,1,1)
        yhat = model(x).item()                  # scalar
        preds.append(yhat)
        w = torch.cat([w[1:], torch.tensor([yhat], dtype=w.dtype)])
    return torch.tensor(preds, dtype=torch.float32)


In [ ]:
# last_window from the end of your *scaled* train+val data:
last_window = train_data_tensor[-30:]           # (30,)
preds_scaled = forecast_rolling(model, last_window, steps=len(data_test))
# inverse-transform to original price units
# inv_preds = scaler.inverse_transform(preds_scaled.view(-1,1)).ravel()
